In [ ]:
# CI_2_SKEW METER (-1,1)

In [ ]:
#This comment is for normality test
#It's possible that your data aren't that far from normal,
#but that the p-value is low because of the relatively large sample size.
#When the sample size is large, a test is likely to report significant differences from
#the normal distribution even for small deviations from normal.

import numpy as np
import pylab as p
import matplotlib.pyplot as plt
from scipy.stats import skew
import pandas as pd
import seaborn as sns
from statsmodels import robust
from math import exp
import robustats
from scipy import stats
from scipy.stats import normaltest

def boxc(df,c):
    (x_most_normal, lmbda_optimal) = stats.boxcox(df[c])
    df[c]=x_most_normal
    return df

def yeoj(df,c):
    (xti, lmbda) = stats.yeojohnson(df[c])
    df[c]=xti
    return df

def outlier_treatment_CI1_S1(df,cols):
    def remove_outlier_Tuckeys(df, cols): #Tuckey's Method
        df = df.copy()
        for c in cols:
            quartile_1, quartile_3 = np.percentile(df[c], [25, 75])
            iqr = quartile_3 - quartile_1
            lower_bound = quartile_1 - (iqr * 3)
            upper_bound = quartile_3 + (iqr * 3)
            df.loc[(df[c] < lower_bound), c] = lower_bound
            df.loc[(df[c] > upper_bound), c] = upper_bound
        return df
    def remove_outlier_MADe(df, cols): #MADe Method
        df = df.copy()
        for c in cols:
            Q2=df[c].quantile(0.50)
            MADe=robust.mad(df[c])
            lower_bound = Q2 -(3*MADe)
            upper_bound = Q2 +(3*MADe)
            df.loc[(df[c] < lower_bound), c] = lower_bound
            df.loc[(df[c] > upper_bound), c] = upper_bound
        return df
    def remove_outlier_median(df, cols):  #Median Method
        df = df.copy()
        for c in cols:
            quartile_1,quartile_2,quartile_3 = np.percentile(df[c], [25,50,75])
            iqr = quartile_3 - quartile_1
            lower_bound = quartile_2 - (iqr * 2.3)
            upper_bound = quartile_2 + (iqr * 2.3)
            df.loc[(df[c] < lower_bound), c] = lower_bound
            df.loc[(df[c] > upper_bound), c] = upper_bound
        return df
    def remove_outlier_adjs(df, cols): #Adjacent Box Method
        df = df.copy()
        for c in cols:
            Q2=df[c].quantile(0.50)
            Q1=df[c].quantile(0.25)
            Q3=df[c].quantile(0.75)
            IQR=Q3-Q1
            list1 = df[c].tolist()
            MC = robustats.medcouple(list1)
            if MC > 0:
                    lower_bound = Q1-1.5 * exp(-3.5*MC) * IQR
                    upper_bound = Q3+1.5 * exp (4*MC) * IQR
            else:
                    lower_bound =Q1-1.5 * exp(-4*MC) * IQR
                    upper_bound = Q3+1.5 * exp (3.5*MC) * IQR
            df.loc[(df[c] < lower_bound), c] = lower_bound
            df.loc[(df[c] > upper_bound), c] = upper_bound
        return df

def CI_2_SK1(df,cols):
    df1 = df.copy()
    for c in cols:
            df1 = df.copy()
            a = df1.skew()   #Store the value of skewness in each column as a panda series
    for c in cols:
                if(a[c])>0.5 or a[c]< (-0.5): # Change the check of skewness accordingly
                        k=[]
                        k=[c]
                        print("Adj Box Plot on {}".format(c))
                        df1=remove_outlier_adjs(df1,k)
                        print("Performed only on column {} of list {}".format(c,k))
                else:
                    print("Checking for normality of {}".format(c))
                    stat, p = normaltest(df1[c])
                    print('Statistics=%.3f, p=%.3f' % (stat, p))
                    # interpret
                    alpha = 0.05
                    S=True
                    if p > alpha:
                            k=[]
                            k=[c]
                            print('Sample looks Gaussian (fail to reject H0)')
                            S=True
                            print("Select the method that you want to use on {}:".format(c))
                            print("1)MADe Method\n2)Median Rule\n3)Tuckey's Method")
                            n=int(input())
                            S=True
                            while(S==True):
                                k=[]
                                k=[c]
                                if (n==1):
                                        df1=remove_outlier_MADe(df1, k)
                                        print("Performed only on column {} of list {}".format(c,k))
                                        S= False
                                elif (n==2):
                                        df1=remove_outlier_median(df1, k)
                                        print("Performed only on column {} of list {}".format(c,k))
                                        S=False
                                elif(n==3):
                                        df1=remove_outlier_Tuckeys(df1, k)
                                        print("Performed only on column {} of list {}".format(c,k))
                                        S=False
                                else:
                                        print("Please select between 1,2 and 3.")
                                        n=int(input())
                    else:
                            print('Sample does not look Gaussian (reject H0)')
                            m=df1[c].min()
                            if ((m)>0):
                                print('BoxCox used on {}'.format(c))
                                df1=boxc(df1,c)
                                seek=df1[c].skew()
                                if (seek>0.5 or seek<(-0.5)):
                                    k=[]
                                    k=[c]
                                    print("Adj Box Plot on {}".format(c))
                                    df1=remove_outlier_adjs(df1,k)
                                    print("Performed only on column {} of list {}".format(c,k))
                                else:
                                    print("Select the method that you want to use on {}:".format(c))
                                    print("1)MADe Method\n2)Median Rule\n3)Tuckey's Method")
                                    n=int(input())
                                    S=True
                                    while(S==True):
                                        k=[]
                                        k=[c]
                                        if (n==1):
                                                df1=remove_outlier_MADe(df1, k)
                                                print("Performed only on column {} of list {}".format(c,k))
                                                S= False
                                        elif (n==2):
                                                df1=remove_outlier_median(df1, k)
                                                print("Performed only on column {} of list {}".format(c,k))
                                                S=False
                                        elif(n==3):
                                                df1=remove_outlier_Tuckeys(df1, k)
                                                print("Performed only on column {} of list {}".format(c,k))
                                                S=False
                                        else:
                                                print("Please select between 1,2 and 3:")
                                                n=int(input())

                            else:
                                print('YeoJohnson used on {}'.format(c))
                                df1=yeoj(df1,c)
                                seek=df1[c].skew()
                                if (seek>0.5 or seek<(-0.5)):
                                    k=[]
                                    k=[c]
                                    print("Adj Box Plot on {}".format(c))
                                    df1=remove_outlier_adjs(df1,k)
                                    print("Performed only on column {} of list {}".format(c,k))
                                else:
                                    print("Select the method that you want to use on {}:".format(c))
                                    print("1)MADe Method\n2)Median Rule\n3)Tuckey's Method")
                                    n=int(input())
                                    S=True
                                    while(S==True):
                                        k=[]
                                        k=[c]
                                        if (n==1):
                                                df1=remove_outlier_MADe(df1, k)
                                                print("Performed only on column {} of list {}".format(c,k))
                                                S= False
                                        elif(n==2):
                                                df1=remove_outlier_median(df1, k)
                                                print("Performed only on column {} of list {}".format(c,k))
                                                S=False
                                        elif(n==3):
                                                df1=remove_outlier_Tuckeys(df1, k)
                                                print("Performed only on column {} of list {}".format(c,k))
                                                S=False
                                        else:
                                                print("Please select between 1,2and 3: ")
                                                n=int(input())

    return df1

# CI_2_SKEW METER(-0.5,0.5)

In [ ]:
#This comment is for normality test
#It's possible that your data aren't that far from normal,
#but that the p-value is low because of the relatively large sample size.
#When the sample size is large, a test is likely to report significant differences from
#the normal distribution even for small deviations from normal.

import numpy as np
import pylab as p
import matplotlib.pyplot as plt
from scipy.stats import skew
import pandas as pd
import seaborn as sns
from statsmodels import robust
from math import exp
import robustats
from scipy import stats
from scipy.stats import normaltest


def boxc(df,c):
    (x_most_normal, lmbda_optimal) = stats.boxcox(df[c])
    df[c]=x_most_normal
    return df

def yeoj(df,c):
    (xti, lmbda) = stats.yeojohnson(df[c])
    df[c]=xti
    return df

def outlier_treatment_CI1_S1(df,cols):
    def remove_outlier_Tuckeys(df, cols): #Tuckey's Method
        df = df.copy()
        for c in cols:
            quartile_1, quartile_3 = np.percentile(df[c], [25, 75])
            iqr = quartile_3 - quartile_1
            lower_bound = quartile_1 - (iqr * 3)
            upper_bound = quartile_3 + (iqr * 3)
            df.loc[(df[c] < lower_bound), c] = lower_bound
            df.loc[(df[c] > upper_bound), c] = upper_bound
        return df
    def remove_outlier_MADe(df, cols): #MADe Method
        df = df.copy()
        for c in cols:
            Q2=df[c].quantile(0.50)
            MADe=robust.mad(df[c])
            lower_bound = Q2 -(3*MADe)
            upper_bound = Q2 +(3*MADe)
            df.loc[(df[c] < lower_bound), c] = lower_bound
            df.loc[(df[c] > upper_bound), c] = upper_bound
        return df
    def remove_outlier_median(df, cols):  #Median Method
        df = df.copy()
        for c in cols:
            quartile_1,quartile_2,quartile_3 = np.percentile(df[c], [25,50,75])
            iqr = quartile_3 - quartile_1
            lower_bound = quartile_2 - (iqr * 2.3)
            upper_bound = quartile_2 + (iqr * 2.3)
            df.loc[(df[c] < lower_bound), c] = lower_bound
            df.loc[(df[c] > upper_bound), c] = upper_bound
        return df
    def remove_outlier_adjs(df, cols): #Adjacent Box Method
        df = df.copy()
        for c in cols:
            Q2=df[c].quantile(0.50)
            Q1=df[c].quantile(0.25)
            Q3=df[c].quantile(0.75)
            IQR=Q3-Q1
            list1 = df[c].tolist()
            MC = robustats.medcouple(list1)
            if MC > 0:
                    lower_bound = Q1-1.5 * exp(-3.5*MC) * IQR
                    upper_bound = Q3+1.5 * exp (4*MC) * IQR
            else:
                    lower_bound =Q1-1.5 * exp(-4*MC) * IQR
                    upper_bound = Q3+1.5 * exp (3.5*MC) * IQR
            df.loc[(df[c] < lower_bound), c] = lower_bound
            df.loc[(df[c] > upper_bound), c] = upper_bound
        return df

def CI_2_SK5(df,cols):
    df1 = df.copy()
    for c in cols:
            df1 = df.copy()
            a = df1.skew()   #Store the value of skewness in each column as a panda series
    for c in cols:
                if(a[c])>1 or a[c]< (-1): # Change the check of skewness accordingly
                        k=[]
                        k=[c]
                        print("Adj Box Plot on {}".format(c))
                        df1=remove_outlier_adjs(df1,k)
                        print("Performed only on column {} of list {}".format(c,k))
                else:
                    print("Checking for normality of {}".format(c))
                    stat, p = normaltest(df1[c])
                    print('Statistics=%.3f, p=%.3f' % (stat, p))
                    # interpret
                    alpha = 0.05
                    S=True
                    if p > alpha:
                            k=[]
                            k=[c]
                            print('Sample looks Gaussian (fail to reject H0)')
                            S=True
                            print("Select the method that you want to use on {}:".format(c))
                            print("1)MADe Method\n2)Median Rule\n3)Tuckey's Method")
                            n=int(input())
                            S=True
                            while(S==True):
                                k=[]
                                k=[c]
                                if (n==1):
                                        df1=remove_outlier_MADe(df1, k)
                                        print("Performed only on column {} of list {}".format(c,k))
                                        S= False
                                elif (n==2):
                                        df1=remove_outlier_median(df1, k)
                                        print("Performed only on column {} of list {}".format(c,k))
                                        S=False
                                elif(n==3):
                                        df1=remove_outlier_Tuckeys(df1, k)
                                        print("Performed only on column {} of list {}".format(c,k))
                                        S=False
                                else:
                                        print("Please select between 1,2 and 3.")
                                        n=int(input())
                    else:
                            print('Sample does not look Gaussian (reject H0)')
                            m=df1[c].min()
                            if ((m)>0):
                                print('BoxCox used on {}'.format(c))
                                df1=boxc(df1,c)
                                seek=df1[c].skew()
                                if (seek>1 or seek<(-1)):
                                    k=[]
                                    k=[c]
                                    print("Adj Box Plot on {}".format(c))
                                    df1=remove_outlier_adjs(df1,k)
                                    print("Performed only on column {} of list {}".format(c,k))
                                else:
                                    print("Select the method that you want to use on {}:".format(c))
                                    print("1)MADe Method\n2)Median Rule\n3)Tuckey's Method")
                                    n=int(input())
                                    S=True
                                    while(S==True):
                                        k=[]
                                        k=[c]
                                        if (n==1):
                                                df1=remove_outlier_MADe(df1, k)
                                                print("Performed only on column {} of list {}".format(c,k))
                                                S= False
                                        elif (n==2):
                                                df1=remove_outlier_median(df1, k)
                                                print("Performed only on column {} of list {}".format(c,k))
                                                S=False
                                        elif(n==3):
                                                df1=remove_outlier_Tuckeys(df1, k)
                                                print("Performed only on column {} of list {}".format(c,k))
                                                S=False
                                        else:
                                                print("Please select between 1,2 and 3:")
                                                n=int(input())

                            else:
                                print('YeoJohnson used on {}'.format(c))
                                df1=yeoj(df1,c)
                                seek=df1[c].skew()
                                if (seek>1 or seek<(-1)):
                                    k=[]
                                    k=[c]
                                    print("Adj Box Plot on {}".format(c))
                                    df1=remove_outlier_adjs(df1,k)
                                    print("Performed only on column {} of list {}".format(c,k))
                                else:
                                    print("Select the method that you want to use on {}:".format(c))
                                    print("1)MADe Method\n2)Median Rule\n3)Tuckey's Method")
                                    n=int(input())
                                    S=True
                                    while(S==True):
                                        k=[]
                                        k=[c]
                                        if (n==1):
                                                df1=remove_outlier_MADe(df1, k)
                                                print("Performed only on column {} of list {}".format(c,k))
                                                S= False
                                        elif(n==2):
                                                df1=remove_outlier_median(df1, k)
                                                print("Performed only on column {} of list {}".format(c,k))
                                                S=False
                                        elif(n==3):
                                                df1=remove_outlier_Tuckeys(df1, k)
                                                print("Performed only on column {} of list {}".format(c,k))
                                                S=False
                                        else:
                                                print("Please select between 1,2and 3: ")
                                                n=int(input())

    return df1

# CI_1_SKEW METER(-1,1)

In [ ]:

from statsmodels import robust
import robustats
from math import exp
import numpy as np
import pylab as p
import matplotlib.pyplot as plt
from scipy.stats import skew
import pandas as pd
import seaborn as sns

def outlier_treatment_CI1_S1(df,cols):
    def remove_outlier_Tuckeys(df, cols): #Tuckey's Method
        df = df.copy()
        for c in cols:
            quartile_1, quartile_3 = np.percentile(df[c], [25, 75])
            iqr = quartile_3 - quartile_1
            lower_bound = quartile_1 - (iqr * 3)
            upper_bound = quartile_3 + (iqr * 3)
            df.loc[(df[c] < lower_bound), c] = lower_bound
            df.loc[(df[c] > upper_bound), c] = upper_bound
        return df
    def remove_outlier_MADe(df, cols): #MADe Method
        df = df.copy()
        for c in cols:
            Q2=df[c].quantile(0.50)
            MADe=robust.mad(df[c])
            lower_bound = Q2 -(3*MADe)
            upper_bound = Q2 +(3*MADe)
            df.loc[(df[c] < lower_bound), c] = lower_bound
            df.loc[(df[c] > upper_bound), c] = upper_bound
        return df
    def remove_outlier_median(df, cols):  #Median Method
        df = df.copy()
        for c in cols:
            quartile_1,quartile_2,quartile_3 = np.percentile(df[c], [25,50,75])
            iqr = quartile_3 - quartile_1
            lower_bound = quartile_2 - (iqr * 2.3)
            upper_bound = quartile_2 + (iqr * 2.3)
            df.loc[(df[c] < lower_bound), c] = lower_bound
            df.loc[(df[c] > upper_bound), c] = upper_bound
        return df
    def remove_outlier_adjs(df, cols): #Adjacent Box Method
        df = df.copy()
        for c in cols:
            Q2=df[c].quantile(0.50)
            Q1=df[c].quantile(0.25)
            Q3=df[c].quantile(0.75)
            IQR=Q3-Q1
            list1 = df[c].tolist()
            MC = robustats.medcouple(list1)
            if MC > 0:
                    lower_bound = Q1-1.5 * exp(-3.5*MC) * IQR
                    upper_bound = Q3+1.5 * exp (4*MC) * IQR
            else:
                    lower_bound =Q1-1.5 * exp(-4*MC) * IQR
                    upper_bound = Q3+1.5 * exp (3.5*MC) * IQR
            df.loc[(df[c] < lower_bound), c] = lower_bound
            df.loc[(df[c] > upper_bound), c] = upper_bound
        return df
    df1 = df.copy()
    for c in cols:
            df1 = df.copy()
            a = df1.skew()   #Store the value of skewness in each column as a panda series
    for c in cols:
                if(a[c])>1 or a[c]< (-1): # Change the check of skewness accordingly
                        print("Adj Box Plot on {}".format(c))
                        k=[]
                        k=[c]
                        print(k)
                        df1=remove_outlier_adjs(df1,k)
                        print("Performed only on column {} of list {}".format(c,k))
                else:
                        print("Select the method that you want to use on {}:".format(c))
                        print("1)MADe Method\n2)Median Rule\n3)Tuckey's Method")
                        n=int(input())
                        S=True
                        while(S==True):
                            if (n==1):
                                    k=[]
                                    k=[c]
                                    df1=remove_outlier_MADe(df1, k)
                                    print("Performed only on column {} of list {}".format(c,k))
                                    S= False
                            elif (n==2):
                                    k=[]
                                    k=[c]
                                    df1=remove_outlier_median(df1, k)
                                    print("Performed only on column {} of list {}".format(c,k))
                                    S=False
                            elif(n==3):
                                    k=[]
                                    k=[c]
                                    df1=remove_outlier_Tuckeys(df1, k)
                                    print("Performed only on column {} of list {}".format(c,k))
                                    print("Tuckey on {}".format(c))
                                    S=False
                            else:
                                    print("Please select between 1,2 and 3.")
                                    n=int(input())
    return df1

# CI_1_SKEW METER(-0.5,0.5)

In [ ]:
from statsmodels import robust
import robustats
from math import exp
import numpy as np
import pylab as p
import matplotlib.pyplot as plt
from scipy.stats import skew
import pandas as pd
import seaborn as sns

def outlier_treatment_CI1_S5(df,cols):
    def remove_outlier_Tuckeys(df, cols): #Tuckey's Method
        df = df.copy()
        for c in cols:
            quartile_1, quartile_3 = np.percentile(df[c], [25, 75])
            iqr = quartile_3 - quartile_1
            lower_bound = quartile_1 - (iqr * 3)
            upper_bound = quartile_3 + (iqr * 3)
            df.loc[(df[c] < lower_bound), c] = lower_bound
            df.loc[(df[c] > upper_bound), c] = upper_bound
        return df
    def remove_outlier_MADe(df, cols): #MADe Method
        df = df.copy()
        for c in cols:
            Q2=df[c].quantile(0.50)
            MADe=robust.mad(df[c])
            lower_bound = Q2 -(3*MADe)
            upper_bound = Q2 +(3*MADe)
            df.loc[(df[c] < lower_bound), c] = lower_bound
            df.loc[(df[c] > upper_bound), c] = upper_bound
        return df
    def remove_outlier_median(df, cols):  #Median Method
        df = df.copy()
        for c in cols:
            quartile_1,quartile_2,quartile_3 = np.percentile(df[c], [25,50,75])
            iqr = quartile_3 - quartile_1
            lower_bound = quartile_2 - (iqr * 2.3)
            upper_bound = quartile_2 + (iqr * 2.3)
            df.loc[(df[c] < lower_bound), c] = lower_bound
            df.loc[(df[c] > upper_bound), c] = upper_bound
        return df
    def remove_outlier_adjs(df, cols): #Adjacent Box Method
        df = df.copy()
        for c in cols:
            Q2=df[c].quantile(0.50)
            Q1=df[c].quantile(0.25)
            Q3=df[c].quantile(0.75)
            IQR=Q3-Q1
            list1 = df[c].tolist()
            MC = robustats.medcouple(list1)
            if MC > 0:
                    lower_bound = Q1-1.5 * exp(-3.5*MC) * IQR
                    upper_bound = Q3+1.5 * exp (4*MC) * IQR
            else:
                    lower_bound =Q1-1.5 * exp(-4*MC) * IQR
                    upper_bound = Q3+1.5 * exp (3.5*MC) * IQR
            df.loc[(df[c] < lower_bound), c] = lower_bound
            df.loc[(df[c] > upper_bound), c] = upper_bound
        return df
    df1 = df.copy()
    for c in cols:
            df1 = df.copy()
            a = df1.skew()   #Store the value of skewness in each column as a panda series
    for c in cols:
                if((a[c])>0.5 or a[c]< (-0.5)): # Change the check of skewness accordingly
                        print("Adj Box Plot on {}".format(c))
                        k=[]
                        k=[c]
                        print(k)
                        df1=remove_outlier_adjs(df1,k)
                        print("Performed only on column {} of list {}".format(c,k))
                else:
                        print("Select the method that you want to use on {}:".format(c))
                        print("1)MADe Method\n2)Median Rule\n3)Tuckey's Method")
                        n=int(input())
                        S=True
                        while(S==True):
                            if (n==1):
                                    k=[]
                                    k=[c]
                                    df1=remove_outlier_MADe(df1, k)
                                    print("Performed only on column {} of list {}".format(c,k))
                                    S= False
                            elif (n==2):
                                    k=[]
                                    k=[c]
                                    df1=remove_outlier_median(df1, k)
                                    print("Performed only on column {} of list {}".format(c,k))
                                    S=False
                            elif(n==3):
                                    k=[]
                                    k=[c]
                                    df1=remove_outlier_Tuckeys(df1, k)
                                    print("Performed only on column {} of list {}".format(c,k))
                                    print("Tuckey on {}".format(c))
                                    S=False
                            else:
                                    print("Please select between 1,2 and 3.")
                                    n=int(input())
    return df1